In [ ]:
!pip install ultralytics opencv-python

In [ ]:
from ultralytics import YOLO
import cv2
import math
import random
from google.colab import drive
from google.colab.patches import cv2_imshow



In [ ]:
drive.mount('/content/drive')

MODEL_PATH = "bestest.pt"
VIDEO_PATH = "video2.mp4"

model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)



In [5]:
def get_gps():
    lat = 28.6139 + random.uniform(-0.00005, 0.00005)
    lon = 77.2090 + random.uniform(-0.00005, 0.00005)
    return lat, lon

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))


In [6]:
# track_id → { gps: [(lat,lon), ...], frames: int }
pothole_dict = {}


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ✅ YOLO + TRACKING
    results = model.track(
        frame,
        tracker="bytetrack.yaml",
        persist=True,
        conf=0.65
    )[0]

    if results.boxes is not None and results.boxes.id is not None:
        boxes = results.boxes.xyxy.cpu().numpy()
        ids = results.boxes.id.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        confs = results.boxes.conf.cpu().numpy()

        for box, track_id, cls, conf in zip(boxes, ids, classes, confs):
            if int(cls) == 0:  # pothole class
                x1, y1, x2, y2 = map(int, box)

                # Draw box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

                label = f"POTHOLE ID {int(track_id)} | {conf:.2f}"
                cv2.putText(frame, label, (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

                lat, lon = get_gps()

                # If new track ID → new pothole
                if track_id not in pothole_dict:
                    pothole_dict[track_id] = {
                        "gps": [(lat, lon)],
                        "frames": 1
                    }
                    print(f"✅ NEW POTHOLE → ID {int(track_id)} @ {lat},{lon}")
                else:
                    pothole_dict[track_id]["gps"].append((lat, lon))
                    pothole_dict[track_id]["frames"] += 1

    cv2_imshow(frame)

cap.release()
cv2.destroyAllWindows()


In [ ]:
print("\nFINAL UNIQUE POTHOLES:\n")
for pid, data in pothole_dict.items():
    avg_lat = sum(g[0] for g in data["gps"]) / len(data["gps"])
    avg_lon = sum(g[1] for g in data["gps"]) / len(data["gps"])
    print(f"ID {int(pid)} | Frames: {data['frames']} | Avg GPS: ({avg_lat}, {avg_lon})")